# Imports & Settings


In [74]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re 
from math import isnan
import wandb
import random
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
# utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

import pickle
from pytorch_tabnet.tab_model import TabNetClassifier

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [75]:
# dtype={'type': str} prevents being confused with data type for large data sets
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})
test_prep = pd.read_csv('data/test_prepared.csv', index_col='id', dtype={'type': str})
train_prep = pd.read_csv('data/train_prepared.csv', index_col='id', dtype={'type': str})

# combine models via hard voting


In [76]:
xgb = XGBClassifier()
xgb.load_model('models/xg/xgboost_full.json')
rf = pickle.load(open('./models/rf/first_try', 'rb'))
nn = TabNetClassifier()
nn.load_model('models/nn/first_try.zip')

/home/till/.local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [77]:
#data = train_prep.copy()
data = test_prep.copy()

In [78]:
features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)

In [79]:
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)
X_test = data.drop('type', axis=1)

In [80]:
results = pd.DataFrame()
results['id'] = X_test.index
#results['type'] = y_test

results['rf'] = rf.predict(X_test)
results['nn'] = nn.predict(X_test.values)
results['xg'] = xgb.predict(X_test)



In [81]:
results.set_index('id', inplace=True)

In [82]:
results['filter'] = [-1] * len(results)
results['indi'] = [-1] * len(results)
results['save'] = [-1] * len(results)

In [83]:
results['emb'] = [-1] * len(results)

In [84]:
boost_emb = XGBClassifier()
boost_emb.load_model('models/nlp/xgboost.json')

#df = pd.read_csv('data/train_curie.csv', index_col='id', dtype={'type': str})
df = pd.read_csv('data/test_curie.csv', index_col='id', dtype={'type': str})
df['curie_similarity'] = df.curie_similarity.apply(eval).apply(np.array)

features = list(df.curie_similarity.values)
"""
labels = df.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)
X_train, emb_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)
"""


'\nlabels = df.type\n# at least xgboost cannot deal with string labels\nlabel_encoder = LabelEncoder()\nlabel_encoder = label_encoder.fit(labels)\nlabels = label_encoder.transform(labels)\nX_train, emb_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)\n'

In [85]:
pred = boost_emb.predict(features)

In [86]:
df['pred'] = pred

In [87]:
for index, item in df.iterrows():
        if index in results.index:
            results.loc[index].emb = item.pred
        

In [88]:
types = ['photo', 'photo negative', 'photographic negative, photographic negative', 'photographic material','digital image',\
'archaeological find',\
'graphics', 'drawing', 'design/drawing/sketch','caricature','slide',\
'poster','plan', 'paper','notes', 'document', 'certificate',\
'medal', 'coin', 'label/sign',\
'manuscript','script, song/vocal music', 'music sheet', 'musical instrument', 'manuscript, musical composition', 'manuscript, sheet music',\
'postcard', 'photo, postcard', 'letter, postcard',\
'letter','letter of honor/honorary address',\
       'seal', 'seal/imprint',\
        'printed notes', 'small print',\
        'packaging', 'crate/box',\
        'audio recording', 'telegram',\
       'invitation',  'calendar',\
       'book','magazines', 'album', 'newspaper', 'folder/booklet',\
       'country',\
       'bag', 'suit', 'doll', 'sheet/linen', 'dish/vessel','jewel', 'tape/ribbon',\
       'sculpture']

In [89]:
# categorize types
types = [
'sculpture',\
'bag', 'suit', 'doll', 'sheet/linen', 'dish/vessel','jewel', 'tape/ribbon',\
'country',\
'paper','book','magazines', 'album', 'newspaper', 'folder/booklet',\
'invitation',  'calendar',\
'audio recording', 'telegram',\
'packaging', 'crate/box',\
'printed notes', 'small print',\
'seal', 'seal/imprint',\
'letter','letter of honor/honorary address',\
'postcard', 'photo, postcard', 'letter, postcard',\
'manuscript','script, song/vocal music', 'music sheet', 'musical instrument', 'manuscript, musical composition', 'manuscript, sheet music',\
'medal', 'coin', 'label/sign',\
'poster','plan','notes', 'document', 'certificate',\
'graphics', 'drawing', 'design/drawing/sketch','caricature','slide',\
'archaeological find',\
'photo', 'photo negative', 'photographic negative, photographic negative', 'photographic material','digital image'
]

In [90]:
def filtering(text):
    pred = []
    for type in types:            
        if type in text:
            pred.append(type)
    if ('drawing' in text) or ('sketch' in text) or ('design' in text):
        pred.append('design/drawing/sketch')
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1


In [91]:
type_indicators = {}
with open('data/type_ind_cut.txt', 'r') as f:
    for line in f:
        a = line.split('\'')
        type = a[1]
        indicators = a[2].split()
        type_indicators[type] = indicators
save_indicators = {}
with open('data/save_indicators.txt', 'r') as f:
    for line in f:
        a = line.split('\'')
        type = a[1]
        indicators = a[2].split()
        save_indicators[type] = indicators

In [92]:
def indicating(text):
    pred = []
    for type in types:
        for indicator in type_indicators[type]:
            if indicator in text:
                pred.append(type)
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1

In [ ]:
def save_indicating(text):
    pred = []
    for type in types:
        for indicator in save_indicators[type]:
            if indicator in text:
                pred.append(type)
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1

In [93]:
df['filter'] = df.text_features.apply(filtering)
df['indicating'] = df.text_features.apply(indicating)
df['save'] = df.text_features.apply(save_indicating)

In [94]:
type_lookup = pd.read_csv('data/type_lookup.csv')

In [95]:
df['filter'] = df['filter'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)
df['indicating'] = df['indicating'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)
df['save'] = df['save'].apply(lambda x: type_lookup[type_lookup.english == x].index[0] if x != -1 else -1)

In [96]:
for index, item in df.iterrows():
        if index in results.index:
            results.loc[index].filter = item['filter']
            results.loc[index].indi = item['indicating']
            results.loc[index].save = item['save']

In [97]:
results.head()

,rf,nn,xg,words,emb
id,,,,,
2652198,36,36,36,-1,-1
3851731,22,22,22,-1,1
1211338,52,52,52,8,31
231244,21,21,21,45,50
2523607,8,8,8,8,8


# evalaluate

In [98]:
from statistics import mode
def vote(predictions):
    if predictions[0] != -1:
        return predictions[0]
    if -1 in predictions: predictions.remove(-1)
    return mode(predictions)

In [99]:
results['prediction'] = results.apply(lambda row: vote([row.save,row.xg,row.rf,row.words,row.emb]), axis=1)

In [100]:
#accuracy_score(results.type, results.prediction)
#0.9038095238095238
#0.9157142857142857
#0.9130952380952381
#0.9145238095238095

# submission

In [101]:
results.head()

,rf,nn,xg,words,emb,prediction
id,,,,,,
2652198,36,36,36,-1,-1,36
3851731,22,22,22,-1,1,22
1211338,52,52,52,8,31,52
231244,21,21,21,45,50,21
2523607,8,8,8,8,8,8


In [102]:
submission = pd.DataFrame({'id': results.index ,'type': results.prediction})

In [103]:
type_lookup = pd.read_csv('data/type_lookup.csv')

In [104]:
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())

In [105]:
submission.to_csv('submissions/voter_with_text_ind.csv', index=False)